# Steps to reproduce experiments in :
## **Reproducible evaluation of diffusion MRI features forautomatic classification of patients with Alzheimer’s disease**  using diffusion MRI from ADNI dataset

## Prealable

The original [ADNI datase](http://adni.loni.usc.edu/) data should be downloaded without further touch (Data we used in our paper was downloaded in October 2016).
Fix the paths where your data is stored on your computer:

In [ ]:
!export ADNI_PATH="~/Aramis/Data/ADNI"

In [ ]:
!export OUT_PATH="~/Aramis/Data/OUTPUT"

In [ ]:
!export WORKING_DIR="~/Aramis/Data/tmp/WORKING_DIR"

## 1. Convert the original ADNI data into BIDS format

In [ ]:
!clinica convert adni-to-bids $ADNI_PATH/IMAGES $ADNI_PATH/CLINICAL_DATA $OUT_PATH/ADNI/BIDS -m [DWI, T1]

Define folders for the next steps:

In [ ]:
import os

dwi_bids = os.path.join(os.environ.get('OUT_PATH'), 'ADNI/BIDS/DWI')
t1w_bids = os.path.join(os.environ.get('OUT_PATH'), 'ADNI/BIDS/T1')
output_dir = os.path.join(os.environ.get('OUT_PATH'), 'TSV')
caps_dir = os.path.join(os.environ.get('OUT_PATH'), 'ADNI/CAPS')
working_dir = os.environ.get('WORKING_DIR')

## 2. Create the subjects lists
Choose the subjects available at baseline for diffusion MRI:

In [ ]:
from Code.subjects_lists.subject_lists import create_subjects_lists

# for AD
create_subjects_lists(dwi_bids, t1w_bids, output_dir, label='AD')
# for CN
create_subjects_lists(dwi_bids, t1w_bids, output_dir, label='CN')
# for MCI
create_subjects_lists(dwi_bids, t1w_bids, output_dir, label='MCI')

## 3. Create the demographic table information
Get demographic information based on the chosen population:

In [ ]:
from Code.subjects_lists.lists_stats_dMRI import statistics_cn_ad_mci_M00

statistics_cn_ad_mci_M00(dwi_bids, output_dir)

## 4. Run image processing pipelines
The fist two pipelines were partly integrated into Clinica software and a postprocesing python script was also developed.

Running bash script `Code/5-ADNI_preprocessing.sh`

In [ ]:
## for diffusion MRI
!clinica run dwi-preprocessing-using-t1 $OUT_PATH/ADNI/BIDS $OUT_PATH/ADNI/CAPS y 0.14 -tsv $OUT_PATH/TSV/subjects_T1_dwi.tsv -wd '/WORKING/DIR' -np 8

In [ ]:
## for T1w MRI
!clinica run t1-volume-new-template $OUT_PATH/ADNI/BIDS $OUT_PATH/ADNI/CAPS 'group-ADNIbl' -tsv $OUT_PATH/TSV/subjects_T1_dwi.tsv -wd '/WORKING/DIR' -np 8

Running bash script `Code/6-ADNI_processing.sh`

In [ ]:
## for diffusion MRI
!clinica run dwi-processing-dti $OUT_PATH/ADNI/CAPS -tsv $OUT_PATH/TSV/subjects_T1_dwi.tsv -wd $WORKING_DIR -np 8

Running post-processing (inspired from `Code/7-ADNI_postprocessing.py`)

In [ ]:
from Code.clinica_ml_dwi.dwi_postprocessing_dti import dwi_postprocessing_dti

participant_tsv_file = 'subjects_T1_dwi.tsv'
tsv = os.path.join(output_dir, participant_tsv_file)
tissue_lists = [[1], [2], [1, 2]]
mask_threshold = [0.3]
smooth = [8]

for i in tissue_lists:
    for j in smooth:
        for k in mask_threshold:
            wf = dwi_postprocessing_dti(caps_dir, tsv, working_directory=working_dir, 
                                        mask_tissues=i, mask_threshold=k, smooth=j)
            wf.run(plugin='MultiProc', plugin_args={'n_procs': 8})

## 5. Run the classification tasks

Results from the papers are as follows:

### Classification results of original data on T1-weighted and diffusion MRI:
(inspired from 8-ADNI_classification_original_data.py)

In [ ]:
from clinica_ml_dwi.mlworkflow_dwi_utils import run_voxel_based_classification

# ########################
# ### Original classification
# ########################

caps_directory= caps_dir
output_dir = os.path.join(os.environ.get('OUT_PATH'), 'classification')
n_threads = 72
n_iterations = 250
test_size = 0.2
grid_search_folds = 10
modality = 'T1'

# ########################
# ### DWI
# ########################

######  CN vs AD
task='AD_vs_CN_VB'
diagnoses_tsv = 'PATH/TO/DIAGONISIS_TSV'
subjects_visits_tsv = 'PATH/TO/SUBJECTS_VISITS_TSV'
run_voxel_based_classification(caps_directory, diagnoses_tsv, subjects_visits_tsv, output_dir,
                                task, n_threads, n_iterations, test_size, grid_search_folds)

#####  CN vs pMCI
task='CN_vs_pMCI_VB'
diagnoses_tsv = 'PATH/TO/DIAGONISIS_TSV'
subjects_visits_tsv = 'PATH/TO/SUBJECTS_VISITS_TSV'
run_voxel_based_classification(caps_directory, diagnoses_tsv, subjects_visits_tsv, output_dir,
                                task, n_threads, n_iterations, test_size, grid_search_folds)

######  CN vs MCI
task='CN_vs_MCI_VB'
diagnoses_tsv = 'PATH/TO/DIAGONISIS_TSV'
subjects_visits_tsv = 'PATH/TO/SUBJECTS_VISITS_TSV'
run_voxel_based_classification(caps_directory, diagnoses_tsv, subjects_visits_tsv, output_dir,
                                task, n_threads, n_iterations, test_size, grid_search_folds)

######  sMCI vs pMCI
task='sMCI_vs_pMCI_VB'
diagnoses_tsv = 'PATH/TO/DIAGONISIS_TSV'
subjects_visits_tsv = 'PATH/TO/SUBJECTS_VISITS_TSV'
run_voxel_based_classification(caps_directory, diagnoses_tsv, subjects_visits_tsv, output_dir,
                                task, n_threads, n_iterations, test_size, grid_search_folds)

# ########################
# ### T1
# ########################

######  CN vs AD
task='AD_vs_CN_VB_T1'
diagnoses_tsv = 'PATH/TO/DIAGONISIS_TSV'
subjects_visits_tsv = 'PATH/TO/SUBJECTS_VISITS_TSV'
run_voxel_based_classification(caps_directory, diagnoses_tsv, subjects_visits_tsv, output_dir,
                                task, n_threads, n_iterations, test_size, grid_search_folds, modality=modality)

#####  CN vs pMCI

task='CN_vs_pMCI_VB_T1'
diagnoses_tsv = 'PATH/TO/DIAGONISIS_TSV'
subjects_visits_tsv = 'PATH/TO/SUBJECTS_VISITS_TSV'
run_voxel_based_classification(caps_directory, diagnoses_tsv, subjects_visits_tsv, output_dir,
                                task, n_threads, n_iterations, test_size, grid_search_folds, modality=modality)

######  CN vs MCI
task='CN_vs_MCI_VB_T1'
diagnoses_tsv = 'PATH/TO/DIAGONISIS_TSV'
subjects_visits_tsv = 'PATH/TO/SUBJECTS_VISITS_TSV'
run_voxel_based_classification(caps_directory, diagnoses_tsv, subjects_visits_tsv, output_dir,
                                task, n_threads, n_iterations, test_size, grid_search_folds, modality=modality)

######  sMCI vs pMCI
task='sMCI_vs_pMCI_VB_T1'
diagnoses_tsv = 'PATH/TO/DIAGONISIS_TSV'
subjects_visits_tsv = 'PATH/TO/SUBJECTS_VISITS_TSV'
run_voxel_based_classification(caps_directory, diagnoses_tsv, subjects_visits_tsv, output_dir,
                                task, n_threads, n_iterations, test_size, grid_search_folds, modality=modality)

### Classification results of balanced data on diffusion MRI:
(inspired from `9-ADNI_classification_balanced_data.py`)

In [ ]:
from clinica_ml_dwi.mlworkflow_dwi_utils import run_voxel_based_classification

# ########################
# ### Original classification
# ########################

caps_directory= caps_dir
output_dir = os.path.join(os.environ.get('OUT_PATH'), 'classification')
n_threads = 72
n_iterations = 250
test_size = 0.2
grid_search_folds = 10

# ########################
# ### Balanced classification
# ########################
balanced_down_sample = True

#####  CN vs pMCI
task='CN_vs_pMCI_VB'
diagnoses_tsv = 'PATH/TO/DIAGONISIS_TSV'
subjects_visits_tsv = 'PATH/TO/SUBJECTS_VISITS_TSV'
run_voxel_based_classification(caps_directory, diagnoses_tsv, 
                               subjects_visits_tsv, output_dir,
                               task, n_threads, n_iterations, 
                               test_size, grid_search_folds, 
                               balanced_down_sample=balanced_down_sample)

######  CN vs MCI
task='CN_vs_MCI_VB'
diagnoses_tsv = 'PATH/TO/DIAGONISIS_TSV'
subjects_visits_tsv = 'PATH/TO/SUBJECTS_VISITS_TSV'
run_voxel_based_classification(caps_directory, diagnoses_tsv, 
                               subjects_visits_tsv, output_dir,
                               task, n_threads, n_iterations, 
                               test_size, grid_search_folds, 
                               balanced_down_sample=balanced_down_sample)

######  sMCI vs pMCI
task='sMCI_vs_pMCI_VB'
diagnoses_tsv = 'PATH/TO/DIAGONISIS_TSV'
subjects_visits_tsv = 'PATH/TO/SUBJECTS_VISITS_TSV'
run_voxel_based_classification(caps_directory, diagnoses_tsv, 
                               subjects_visits_tsv, output_dir,
                               task, n_threads, n_iterations, 
                               test_size, grid_search_folds, 
                               balanced_down_sample=balanced_down_sample)

### Classification results of feature selection bias on diffusion MRI:
(inspired from 10-ADNI_classification_feature_selection.py)

In [ ]:
from clinica_ml_dwi.mlworkflow_dwi_utils import run_voxel_based_classification

# ########################
# ### Original classification
# ########################

caps_directory= caps_dir
output_dir = os.path.join(os.environ.get('OUT_PATH'), 'classification')
diagnoses_tsv = 'PATH/TO/DIAGONISIS_TSV'
subjects_visits_tsv = 'PATH/TO/DIAGONISIS_TSV'
n_threads = 72
n_iterations = 250
test_size = 0.2
grid_search_folds = 10
tissue_type=['GM_WM']
task='AD_vs_CN_VB'

# ########################
# ### ANOVA feature selection
# ########################

## Nested ANOVA
feature_selection_method='ANOVA'
run_voxel_based_classification(caps_directory, 
                               diagnoses_tsv, 
                               subjects_visits_tsv, 
                               output_dir,
                               task, 
                               n_threads, 
                               n_iterations, 
                               test_size, 
                               grid_search_folds, 
                               tissue_type=tissue_type, 
                               feature_selection_nested=True, 
                               feature_selection_method=feature_selection_method)

## Nested SVM-RFE
feature_selection_method='RFE'
run_voxel_based_classification(caps_directory, 
                               diagnoses_tsv, 
                               subjects_visits_tsv, 
                               output_dir,
                               task, 
                               n_threads, 
                               n_iterations, 
                               test_size, 
                               grid_search_folds, 
                               tissue_type=tissue_type, 
                               feature_selection_nested=True, 
                               feature_selection_method=feature_selection_method)

## Non_nested ANOVA
feature_selection_method='ANOVA'
run_voxel_based_classification(caps_directory, 
                               diagnoses_tsv, 
                               subjects_visits_tsv, 
                               output_dir,
                               task, 
                               n_threads, 
                               n_iterations, 
                               test_size, 
                               grid_search_folds, 
                               tissue_type=tissue_type, 
                               feature_selection_non_nested=True, 
                               feature_selection_method=feature_selection_method)

## Non_nested SVM-RFE
feature_selection_method='RFE'
run_voxel_based_classification(caps_directory, 
                               diagnoses_tsv, 
                               subjects_visits_tsv, 
                               output_dir,
                               task, 
                               n_threads, 
                               n_iterations, 
                               test_size, 
                               grid_search_folds, 
                               tissue_type=tissue_type, 
                               feature_selection_non_nested=True, 
                               feature_selection_method=feature_selection_method)